# TAREA 2:

1.- Hacer un modelo regresión logistica con woes con la tabla que generaron en la tarea 1.

2.- Hacer un árbol de clasificación con la misma tabla que el problema 1. No olviden que tienen que hacer el ajuste de hiperparámetros.

3.- Seleccionar con validación cruzada cuál es el mejor modelo de los dos problemas anteriores.

4.- Obtener los puntos óptimos de decisión para el modelo seleccionado.

En todos los casos obtener: Gini, KS, ROC en entrenamiento y validación.
Fecha de entrega: Viernes 7 de enero.


## 1.- Modelo de Regresión Logística con Woes